<a href="https://colab.research.google.com/github/amitpanthi/Twitter-Sentiment-Analysis/blob/main/Twitter_Sentiment_Analysis_Streamlit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install streamlit

In [ ]:
%%writefile model.py
import torch
import torch.nn as nn

class SentimentPredictor(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, num_lstm_layers, pad_token):
        super(SentimentPredictor, self).__init__()
        
        self.embedding = nn.Embedding(num_embeddings=input_size, embedding_dim=embedding_size, padding_idx=pad_token)
        self.lstm = nn.LSTM(embedding_size, hidden_size, num_lstm_layers, dropout=0.5)
        self.dense1 = nn.Linear(hidden_size, 64)
        self.dense2 = nn.Linear(64, 1)

    def forward(self, x):
        x = self.embedding(x)
        x, _ = self.lstm(x)
        x = torch.relu(self.dense1(x[-1, :, :])) # get hidden size first
        x = self.dense2(x)

        return x

Writing model.py


In [ ]:
%%writefile predict.py
import torch
from nltk import RegexpTokenizer
from model import SentimentPredictor

class Predictor:
    def __init__(self, pth_file):
        checkpoint = torch.load(pth_file)
        hparams = checkpoint["hparams"]
        model_params = checkpoint["model_params"]
        self.stoi = checkpoint["stoi"]
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'

        self.model = SentimentPredictor(hparams["input_size"], hparams["embedding_size"], 
                                        hparams["hidden_size"], hparams["num_layers"], hparams["pad_token"]).to(self.device)
        self.model.load_state_dict(model_params)


    def get_sentiment(self, tweet):
        self.model.eval()
        tweet = tweet.lower()
        tokenizer = RegexpTokenizer(r"\w+")
        tokens = tokenizer.tokenize(tweet)
        tweet_tokens = [self.stoi.get(token, 0) for token in tokens]
        tweet_tokens = torch.Tensor(tweet_tokens).long().to(self.device)
        tweet_tokens = torch.transpose(tweet_tokens.unsqueeze(0), 0, 1)

        output = self.model(tweet_tokens)
        output = torch.sigmoid(output).item()

        if output >= 0.42 and output <= 0.57:
            return f"Score: {output:.2f}. Neutral"
        elif output > 0.57:
            return f"Score: {output:.2f}. Positive"
        else:
            return f"Score: {output:.2f}. Negative"

Writing predict.py


In [ ]:
%%writefile app.py
import streamlit as st
from predict import Predictor

def main():
    sa_pred = Predictor("sentiment.pth")
    st.set_page_config(page_title="Tweet Sentiment Predictor", page_icon="🤬")
    st.title("Tweet Sentiment Predictor")
    user_input = st.text_input("Enter your tweet here")

    if st.button("Find Sentiment"):
        st.write(f"{sa_pred.get_sentiment(user_input)}")

if __name__ == "__main__":
    main()

Writing app.py


In [ ]:
!pip install pyngrok

In [ ]:
!streamlit run app.py &>/dev/null&

In [ ]:
from pyngrok import ngrok
!ngrok authtoken <your auth token>

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
# Setup a tunnel to the streamlit port 8501
public_url = ngrok.connect(8501)
public_url

In [ ]:
ngrok.kill()